**Special installation for getting alpha release of shapely**

In [ ]:
#conda install -c conda-forge shapely
#conda install -c conda-forge/label/shapely_dev shapely=1.8a1

In [ ]:
# pip install streamstats
# conda install shapely
# pip install hydrofunctions


In [1]:
from streamstats import Watershed
import hydrofunctions as hf
import pandas as pd
import pickle
from shapely.geometry import shape, Point
from shapely.validation import explain_validity
from shapely.validation import make_valid
import numpy as np

I believe a bug exists with shapely contains() function due to the boundary precision being very different from the point precision

In [2]:
#GaugeWatershed is of a GeoJson 

def PointInWatershedBool(GaugeWatershed, pointLat, pointLong):

    # construct point based on lon/lat returned by geocoder
    point = Point(pointLong, pointLat)

    # check each polygon to see if it contains the point
    for feature in GaugeWatershed['features']:
        polygon = shape(feature['geometry'])
        if(polygon.is_valid):
            return polygon.contains(point)
        poly2 = make_valid(polygon)
        return poly2.contains(point)

In [3]:
#issue with converting lat/long to actual distances instead of treating the lat/long as x/y grid
#should still convey correct relative distances over such small degree changes (only 3/7 degrees)
def PointInWatershedDistance(GaugeWatershed, pointLat, pointLong):

    # construct point based on lon/lat returned by geocoder
    point = Point(pointLong, pointLat)

    # check each polygon to see if it contains the point
    for feature in GaugeWatershed['features']:#there should only be one feature since it is a continuous watershed
        polygon = shape(feature['geometry'])
        return polygon.distance(point)

In [ ]:
#collect water gauge site data
site = hf.NWIS( None,  'dv', period='P3649D',parameterCd='00060',  stateCd = 'co',) # '06730200' = site, 'iv' = instantaneous values / 'dv' = daily mean values, 'P55D' = Past 55 days, stateCd='va' => download all of virginia (careful), bBox = boundary box
# From Docs: "The Daily Mean discharge data are great for situations where you want to know the baseflow of a watershed, or the volume of water that is entering a reservoir or if you want to learn about the long-term hydrology of a watershed."
site.meta

In [ ]:
#save site data
afile = open('GaugeSites_CO.pkl', 'wb')
pickle.dump(site, afile)
afile.close()

In [4]:
#read site data
file2 = open('GaugeSites_CO.pkl', 'rb')
site = pickle.load(file2)
file2.close()

In [ ]:
site.meta

In [5]:
#read SnoTel location data
file2 = open("SnoLocations.pkl", 'rb')
SnoLocations = pickle.load(file2)
file2.close()

In [25]:
SnoLocations

{23: (40.43254, -105.73337),
 33: (37.79926, -107.92426),
 36: (37.43087, -108.16954),
 38: (37.47922, -106.8017),
 42: (40.34703, -106.09433),
 45: (39.76487, -107.35681),
 62: (40.03523, -105.54426),
 66: (40.53215, -105.887),
 67: (40.81588, -106.74835),
 84: (39.3035, -106.11307),
 91: (40.80571, -105.76993),
 93: (39.64631, -105.86973),
 94: (40.36783, -106.74038),
 97: (39.29722, -106.60694),
 99: (39.80392, -105.77789),
 107: (40.20105, -105.60248),
 108: (37.65083, -107.80602),
 121: (39.43577, -105.91078),
 137: (40.53397, -106.7813),
 140: (39.56667, -105.8),
 147: (38.28555, -106.37069),
 167: (37.89183, -108.19544),
 169: (37.37929, -106.54835),
 176: (37.61497, -105.37327),
 179: (37.01878, -106.45179),
 181: (39.37991, -106.19681),
 189: (40.3948, -106.60408),
 190: (39.60468, -106.54128),
 194: (37.20945, -105.19959),
 198: (37.51976, -107.32152),
 214: (37.72194, -107.26015),
 217: (38.41795, -108.38249),
 220: (39.87505, -107.59853),
 228: (40.53743, -106.6768),
 236: 

In [17]:

siteCodes = list(site.meta.keys())
spatialDistances_df = pd.DataFrame(index=siteCodes, columns= list(SnoLocations.keys()))
spatialBool_df = pd.DataFrame(index=siteCodes, columns= list(SnoLocations.keys()))
i = 0
for siteCD in siteCodes:
    i+=1
    print("proccessing %d" %i)
    siteMeta = site.meta[siteCD]['siteLatLongSrs']
    #Call streamstats for watershed
    GWatershed = Watershed(lat=siteMeta['latitude'], lon=siteMeta['longitude'])
    #gets GeoJSON from watershed 
    js = GWatershed.boundary
    for SnowSite in SnoLocations:
        spatialBool_df[SnowSite][siteCD] = PointInWatershedBool(js, SnoLocations[SnowSite][0], SnoLocations[SnowSite][1])
        spatialDistances_df[SnowSite][siteCD] = PointInWatershedDistance(js, SnoLocations[SnowSite][0], SnoLocations[SnowSite][1])

proccessing 282
proccessing 283
proccessing 284
proccessing 285
proccessing 286
proccessing 287
proccessing 288
proccessing 289
proccessing 290
proccessing 291
proccessing 292
proccessing 293
proccessing 294
proccessing 295
proccessing 296
proccessing 297
proccessing 298
proccessing 299
proccessing 300
proccessing 301
proccessing 302
proccessing 303
proccessing 304
proccessing 305
proccessing 306
proccessing 307
proccessing 308
proccessing 309
proccessing 310
proccessing 311
proccessing 312
proccessing 313
proccessing 314
proccessing 315
proccessing 316
proccessing 317
proccessing 318
proccessing 319
proccessing 320
proccessing 321
proccessing 322
proccessing 323
proccessing 324
proccessing 325
proccessing 326
proccessing 327
proccessing 328
proccessing 329
proccessing 330
proccessing 331
proccessing 332
proccessing 333
proccessing 334
proccessing 335
proccessing 336
proccessing 337
proccessing 338
proccessing 339
proccessing 340
proccessing 341
proccessing 342
proccessing 343
proccess

In [64]:
#save Spatial boolean matrix data
afile = open('spatialBool_df.pkl', 'wb')
pickle.dump(spatialBool_df, afile)
afile.close()

In [ ]:
#read spatial bool file
file2 = open('spatialBool_df.pkl', 'rb')
spatialBool_df = pickle.load(file2)
file2.close()

In [65]:
#save spatial distance matrix data
afile = open('spatialDistances_df.pkl', 'wb')
pickle.dump(spatialDistances_df, afile)
afile.close()

In [26]:
#read spatial bool file
file2 = open('spatialDistances_df.pkl', 'rb')
spatialDistances_df = pickle.load(file2)
file2.close()

In [66]:
spatialBool_df

,23,33,36,38,42,45,62,66,67,84,...,807,817,818,821,825,828,829,853,854,866
USGS:06613950,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
USGS:06614800,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
USGS:06620000,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
USGS:06659580,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
USGS:06696980,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USGS:394839104570300,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
USGS:401723105400000,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
USGS:401727105400000,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
USGS:401733105392404,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [63]:
spatialBool_df.apply(pd.value_counts) #shows how sparse this matrix is

,23,33,36,38,42,45,62,66,67,84,...,807,817,818,821,825,828,829,853,854,866
False,370,370,372,371,369,373.0,369,367,371,365,...,372,372,368,371,368,371,369,369,372,372
True,3,3,1,2,4,NaN,4,6,2,8,...,1,1,5,2,5,2,4,4,1,1


In [56]:
# find of sites which API had an error on so that just these sites can be ran again
#not a great solution but it works for small reruns 
find_ers_df = spatialBool_df.isna()
for siteCD in siteCodes:
    for SnowSite in SnoLocations:
        if(find_ers_df[SnowSite][siteCD] == True):
            print(siteCD)

USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:07134990
USGS:0

In [52]:
erList = ['USGS:06713000','USGS:06719560','USGS:07100300','USGS:07103980','USGS:07134990','USGS:09041900','USGS:09046600','USGS:09085100','USGS:09124010','USGS:09132940','USGS:09132995','USGS:09361500','USGS:394329104490101']
erList2 = ['USGS:07134990'] #somehow this station is at a point in the US that does not have a HUC

In [55]:
#rerun function for error list of gauge sites
for siteCD in erList:
    print("proccessing ", siteCD)
    siteMeta = site.meta[siteCD]['siteLatLongSrs']
    #Call streamstats for watershed
    GWatershed = Watershed(lat=siteMeta['latitude'], lon=siteMeta['longitude'])
    #gets GeoJSON from watershed 
    js = GWatershed.boundary
    for SnowSite in SnoLocations:
        spatialBool_df[SnowSite][siteCD] = PointInWatershedBool(js, SnoLocations[SnowSite][0], SnoLocations[SnowSite][1])
        spatialDistances_df[SnowSite][siteCD] = PointInWatershedDistance(js, SnoLocations[SnowSite][0], SnoLocations[SnowSite][1])

proccessing  USGS:07134990


In [67]:
spatialDistances_df

,23,33,36,38,42,45,62,66,67,84,...,807,817,818,821,825,828,829,853,854,866
USGS:06613950,0.753193,3.20963,3.64719,3.23393,0.506863,1.2206,1.12251,0.573448,0.158836,1.44529,...,0.642136,3.12477,0.776448,2.10563,0.714895,0.364305,2.21529,3.16633,2.21201,3.65597
USGS:06614800,0.132979,3.36141,3.80325,3.13253,0.239725,1.62778,0.546655,0.0413337,0.917343,1.19385,...,0.873649,3.0771,0.436514,2.44379,0.252716,0.504118,2.03766,3.26606,2.5888,3.24838
USGS:06620000,0.132979,2.89446,3.33442,2.88961,0.00389194,0.965295,0.512112,0.0105219,0.0855012,1.02561,...,0.305858,2.79013,0.36361,1.83544,0.128988,0.00172251,1.83784,2.84154,1.95859,3.1466
USGS:06659580,0.403611,3.67464,4.11716,3.48046,0.541485,1.85004,0.838251,0.295356,0.902359,1.54462,...,1.11701,3.42019,0.0842725,2.70214,0.603495,0.719724,2.38793,3.58876,2.83572,3.57589
USGS:06696980,1.0414,2.45714,2.88227,2.02192,0.943454,1.39214,0.739216,1.11969,1.60326,0.105011,...,0.952967,2.00212,1.50154,1.87881,0.811725,1.02551,0.912478,2.31245,2.05903,2.1741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USGS:394839104570300,0.99844,3.57578,3.98384,2.89239,1.26173,2.40477,0.633127,1.1808,2.05824,1.26632,...,1.73858,2.98081,1.29102,3.01981,1.05264,1.52785,1.88505,3.40722,3.19556,2.13821
USGS:401723105400000,0.149625,3.34473,3.78301,3.01612,0.411826,1.75012,0.27745,0.314831,1.18535,1.06733,...,1.00574,2.98451,0.593786,2.51512,0.241335,0.697527,1.90781,3.23034,2.67141,3.01097
USGS:401727105400000,0.152999,3.36149,3.79977,3.03165,0.42876,1.76869,0.283552,0.32309,1.19909,1.0831,...,1.02433,3.00057,0.594227,2.53287,0.259971,0.714823,1.92316,3.24707,2.68936,3.02026
USGS:401733105392404,0.149625,3.33768,3.77583,3.00655,0.411826,1.7495,0.261886,0.314831,1.18535,1.05799,...,1.00574,2.97551,0.590386,2.5112,0.241335,0.697527,1.89808,3.22282,2.66806,2.99688
